In [29]:
# import
import os
from dotenv import load_dotenv

import openai
from langchain import OpenAI
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

import pandas as pd
from datetime import datetime
from feast import FeatureStore
from pprint import pprint

In [30]:
# set OpenAI API key
load_dotenv(dotenv_path="../../../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")
# set OpenAI API Key to langchain and set temperature to 0.9
llm = OpenAI(openai_api_key=openai.api_key, temperature=0.9)

In [31]:
# Load Feast Store
feast_repo_path = "./feature_store_feast/feature_repo"
store = FeatureStore(repo_path=feast_repo_path)

In [32]:
# Following Feast Readme
pd.read_parquet(feast_repo_path+"/data/driver_stats.parquet")

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2023-06-16 22:00:00+00:00,1005,0.015975,0.837928,584,2023-07-01 22:42:22.569
1,2023-06-16 23:00:00+00:00,1005,0.231635,0.732077,790,2023-07-01 22:42:22.569
2,2023-06-17 00:00:00+00:00,1005,0.702013,0.975178,209,2023-07-01 22:42:22.569
3,2023-06-17 01:00:00+00:00,1005,0.282054,0.711458,662,2023-07-01 22:42:22.569
4,2023-06-17 02:00:00+00:00,1005,0.180257,0.793173,404,2023-07-01 22:42:22.569
...,...,...,...,...,...,...
1802,2023-07-01 20:00:00+00:00,1001,0.882013,0.217291,913,2023-07-01 22:42:22.569
1803,2023-07-01 21:00:00+00:00,1001,0.940722,0.614214,892,2023-07-01 22:42:22.569
1804,2021-04-12 07:00:00+00:00,1001,0.006743,0.003659,805,2023-07-01 22:42:22.569
1805,2023-06-24 10:00:00+00:00,1003,0.623993,0.582351,805,2023-07-01 22:42:22.569


In [33]:
# Build a training dataset in Feast
entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

training_df = store.get_historical_features(
    entity_df=entity_df,
    features = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

print(training_df.head())

# Train model
# model = ml.fit(training_df)

   driver_id           event_timestamp  conv_rate  acc_rate  avg_daily_trips
0       1001 2021-04-12 10:59:42+00:00   0.006743  0.003659              805
1       1002 2021-04-12 08:12:10+00:00   0.674452  0.935988              329
2       1003 2021-04-12 16:40:26+00:00   0.757471  0.637771              499
3       1004 2021-04-12 15:01:12+00:00   0.600042  0.212857              238


In [34]:
# Load Feast feature values into your online store
!cd $feast_repo_path && feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")

Materializing 2 feature views to 2023-07-02 20:03:49+09:00 into the sqlite online store.

driver_hourly_stats from 2023-07-02 19:56:16+09:00 to 2023-07-02 20:03:49+09:00:
0it [00:00, ?it/s]
driver_hourly_stats_fresh from 2023-07-02 19:56:16+09:00 to 2023-07-02 20:03:49+09:00:
0it [00:00, ?it/s]


In [35]:
feature_vector = store.get_online_features(
    features=[
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
    entity_rows=[{"driver_id": 1001}]
).to_dict()

pprint(feature_vector)

# Make prediction
# model.predict(feature_vector)

{'acc_rate': [0.6142140030860901],
 'avg_daily_trips': [892],
 'conv_rate': [0.9407216906547546],
 'driver_id': [1001]}


In [40]:
# Check on Feast UI
# !cd $feast_repo_path && feast ui --port 9999

INFO:     Started server process [1807]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9999 (Press CTRL+C to quit)
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1807]
Traceback (most recent call last):
  File "/Users/godpeny/Code/venv/openai-research/bin/feast", line 8, in <module>
    sys.exit(cli())
             ^^^^^
  File "/Users/godpeny/Code/venv/openai-research/lib/python3.11/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/godpeny/Code/venv/openai-research/lib/python3.11/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/Users/godpeny/Code/venv/openai-research/lib/python3.11/site-packages/click/core.py", line 1657, in invoke
    return _process_resu

In [42]:
# Prompt
template = """Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [43]:
class FeastPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features=[
                "driver_hourly_stats:conv_rate",
                "driver_hourly_stats:acc_rate",
                "driver_hourly_stats:avg_daily_trips",
            ],
            entity_rows=[{"driver_id": driver_id}],
        ).to_dict()
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)

In [44]:
prompt_template = FeastPromptTemplate(input_variables=["driver_id"])

In [45]:
print(prompt_template.format(driver_id=1001))

Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: 0.9407216906547546
Acceptance rate: 0.6142140030860901
Average Daily Trips: 892

Your response:


In [46]:
# Use "prompt_template" from Feast in a Lang Chain
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_template)
chain.run(1001)

"Dear driver,\n\nI hope this message finds you well. I wanted to provide you with an update on your current stats as a driver. Your conversation rate is an impressive 0.9407, which is fantastic! Your ability to engage with passengers and create a positive experience is truly commendable. Keep up the great work!\n\nAdditionally, your acceptance rate stands at 0.6142, showcasing your dedication to accepting ride requests. It's clear that you are committed to serving your passengers and providing them with a reliable service.\n\nLastly, I wanted to mention that your average daily trips amount to 892. This is a remarkable achievement and demonstrates your efficiency as a driver.\n\nKeep up the excellent work, and remember to keep spreading positivity and creating memorable experiences for your passengers! You're doing a fantastic job.\n\nAnd now, to lighten the mood, here's a silly chicken joke for you: Why did the chicken go to the seance? To talk to the other side! Remember, even chicken